In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive

drive_path = '/content/drive'
in_drive_path = '/MyDrive/Casos de analítica/Caso 1/data/'
path = drive_path + in_drive_path

drive.mount(drive_path)

Mounted at /content/drive


# Análisis de datos existentes

Se realiza una exploratoria sobre los datos provistos en el enunciado para responder las primeras preguntas

## Análisis de conjunto de datos Book-Crossing

Esta fuente consiste en 3 datasets que describen:
- Los libros recopilados
- Los usuarios de la plataforma origen
- Las reseñas que estos usuarios hn dado a los libros

Todos están almacenados en formato CSV, con un punto y coma (;) como separador. Requieren el encoding "latin1" para ser leídos correctamente

Se identificó el uso del string 'n/a' para representar valores faltantes, por lo que este y las celdas vacías, o con espacios en blanco, son marcadas como indicadores de esto

In [ ]:
na_values=['n/a', '', ' ']

### Análisis del dataset de libros

El dataset contiene filas inválidas como la siguiente:


![t](https://drive.google.com/uc?export=view&id=1tNfe5r-EhxyyQrIQv4E9lfuoI36TV-Wm)

Donde el token \&amp; hace que se detecte un cambio de columna, por lo cuál este registro tendría una más que el total, por lo que se evitan los errores al encontrar filas malformadas.

In [ ]:
books_df = pd.read_csv(path + 'BX-Books.csv', encoding='latin-1', sep=";", on_bad_lines='warn')

<ipython-input-5-988befe56616>:1: ParserWarning: Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

  books_df = pd.read_csv(path + 'BX-Books.csv', encoding='latin-1', sep=";", on_bad_lines='warn')
<ipython-input-5-988befe56616>:1: ParserWarning: Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

  books_df = pd.read_csv(path + 'BX-Books.csv', encoding='latin-1', sep=";", on_bad_lines='warn')
<ipython-input-5-988befe56616>:1: ParserWarning: Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

  books_df = pd.read_csv(path + 'BX-Books.csv', encoding='latin-1', sep=";", on_bad_lines='warn')
<ipython-input-5-988befe56616>:1: ParserWar

Puede observarse que esto ocurre en varias ocasiones, que son omitidas por el lector. Esto permite cargar la mayoría de registros, pero perdiendo estos 19 registros. El dataset final contiene:

In [ ]:
books_df.shape

(271360, 8)

filas y columnas, respectivamente. Las propiedades son:

In [ ]:
for c in books_df.columns:
  print(c)

ISBN
Book-Title
Book-Author
Year-Of-Publication
Publisher
Image-URL-S
Image-URL-M
Image-URL-L


Al realizar una inspección visual sobre los datos se identifica que las últimas 3 corresponden a una imagen de la portada del libro en distintas calidades.

In [ ]:
books_df.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...



En los registros cargados se identifican algunos valores faltantes, aunque en una cantidad no significativa

In [1]:
def compute_nas(df):
  nas = pd.DataFrame({ 'Cantidad': df.isna().sum()})
  nas['Porciento'] = nas['Cantidad'] / df.shape[0]

  return nas

compute_nas(books_df)

NameError: name 'books_df' is not defined

### Análisis del dataset de usuarios

Este dataset no presenta filas que generen errores de lectura

In [ ]:
users_df = pd.read_csv(path + 'BX-Users.csv', encoding='latin1', sep=";", na_values=na_values)

En total contiene

In [ ]:
users_df.shape

(278858, 3)

filas y columnas, respectivamente. Las segundas son:

In [ ]:
for c in users_df.columns:
  print(c)

User-ID
Location
Age


Mediante inspección visual, sin embargo, se identifica que la columna Location contiene distintos niveles de ubicación anidados

In [ ]:
users_df.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Que podrían proveer más información acerca de la base de usuarios si fueran tratadas como propiedades independientes.

Así mismo, la inspección muestra la existencia de valores faltantes. Al realizar el cálculo correspondiente

In [ ]:
compute_nas(users_df)

,Cantidad,Porciento
User-ID,0,0.000000
Location,0,0.000000
Age,110762,0.397199


Se observa que la edad tiene un alto volúmen de valores faltantes. Sin embargo, la presencia de valores en todos los registros de Location podría no ser indicativa de la información real, pues es posible que en algunos registros no estén informados todos los niveles de ubicación existentes

Se agrupan los niveles en que se definen los registros según la ocurrencia del separador, para analizar las ocurrencias

In [ ]:
def number_of_locations(l):
  return len(l.split(','))

users_df.groupby(users_df['Location'].map(number_of_locations))['Location'].count()

,Location
Location,
1,1
2,1
3,277348
4,1417
5,72
6,11
7,4
8,2
9,2


Puede observarse que, aunque la mayoría de casos mantienen el formato esperado, existen cantidades con mayor nivel de especificidad, y casos extremos que, por su cantidad de incidencias y desviación de lo esperado, es probable sean errores de formato.

Se estudian los casos con estas desviaciones

In [ ]:
users_df[~users_df['Location'].map(number_of_locations).between(3, 5)]['Location'].to_list()

['st. louis, missouri',
 'new delhi, 7/2,block-o,r.k.puram,sector-13,new delhi , india, india',
 'chula vista, tijuana, california, baja california, usa, mexico',
 'isulan,sultan,kudarat, isulan, sultan kudarat, philippines',
 '1,tebuk tukang emas,bagan datoh, perak, n/a, malaysia',
 'istanbul,eskisehir,antalya/olympos, istanbul,eskisehir,antalya/olympos, turkey',
 '90 lawerances, ave apt 2,dorchester,ma 02121, massachusetts, usa',
 'tianjin, room 308,61 men,guishan li,xueyuan rd. hexi dist, china',
 'philippine science high school - cmc, mcc main stadium, sagadan, tubod, lanao del norte, philippines',
 'lawrenceville',
 'ogba,lagos,nigeria, ,ogba,lagosstate, nigeria',
 'dae-jeon, dae-jeon, yu-song, guam-dong, 626-1, 302, south korea',
 'silver spring, md, metropolitan washington, dc area (dc, md, and va), usa',
 'makati city, philippines, pangasinan,,region 3, philippines',
 'bangalore, #341,4main,6cross,2phase,r`nagar,m`nagar,b`lore-10, india',
 'nmrl, addl-ambernath, thane, mumbai, 

Al comprobar los casos identificados con las fuentes, puede inicialmente considerarse que a mayor granularidad se añade un nivel a partir de la población. Nótese en los casos inferiores, donde la localización comienza a construirse desde la localidad, pero no se llega a informar el país. A medida que aumenta la cantidad de niveles aparecen más puntos intermedios entre esta y el país, una vez es reportado.

Sin embargo, para los casos superiores comienzan a incluirse niveles inferiores a la población, como posiblemente direcciones. Esto impide un tratamiento genérico de, por ejemplo, construir tres columnas tomando la ciudad/población (primer elemento), el equivalente a un estado/comunidad (antepenúltimo elemento) y el país (último elemento), al no poderse tomar como criterio la presencia de la población en la posición inicial.

También existe un caso entre los mostrados en que el antepenúltimo nivel repite la información de país (el segundo de la muestra anterior), y otros donde la información intermedia es inconsistente: por ejempo, en el tercer registro de la muestra anterior, se incluyen detalles geográficos de Estados Unidos en una región que al final corresponde, y se indica, como parte de México.

Aunque las inconsistencias citadas aparecen en elementos puntuales, puede también ser el caso de otros registros que por su longitud más cercana a lo normal no se hayan incluido en la muestra.

### Análisis del dataset de reseñas

Este dataset tampoco presenta errores al momento de lectura

In [ ]:
reviews_df = pd.read_csv(path + 'BX-Book-Ratings.csv', encoding='latin1', sep=";", na_values=na_values)

Está compuesto por

In [ ]:
reviews_df.shape

(1149780, 3)

filas y columnas, respectivamente, siendo las últimas

In [ ]:
for c in reviews_df.columns:
  print(c)

User-ID
ISBN
Book-Rating


Las dos primeras columnas hacen referencia a los identificadores de usuarios y libros (el ISBN) de los dataset anteriores. Al observar una muestra

In [ ]:
reviews_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


se determina que las reseñas son solamente el nivel dado al libro, en una escala del 0-10:

In [ ]:
reviews_df['Book-Rating'].min(), reviews_df['Book-Rating'].max()

(0, 10)

Existen reseñas de

In [ ]:
unique_users = pd.DataFrame({ 'User-ID': reviews_df['User-ID'].unique() })
reviewers = unique_users.shape[0]

reviewers

105283

usuarios para un total de

In [ ]:
unique_books = pd.DataFrame({ 'ISBN': reviews_df['ISBN'].unique() })
reviewed_books = unique_books.shape[0]

reviewed_books

340556

libros

Al analizar las relaciones con los conjuntos de datos origen puede observarse que

In [ ]:
users_df.merge(unique_users, on='User-ID', how='inner').shape[0]

105283

todas las claves reportadas en el dataset de calificaciones de los libros corresponden a usuarios registrados, por lo que no hay errores en estas.

Sin embargo, el número de libros existente en el dataset de origen es sensiblemente inferior a los reportados en las reviews

In [ ]:
books_df.merge(unique_books, on=['ISBN'], how='inner').shape[0]

270151

Se considera la posibilidad de ser debido a la existencia de espacios en blanco en las claves, por lo que se repite la revisión eliminando los existentes

In [ ]:
books_df.merge(unique_books, left_on=books_df['ISBN'].str.strip(), right_on=unique_books['ISBN'].str.strip(), how='inner').shape[0]

270154

Lo cuál arroja tres nuevos resultados en el cruce. Esto puede indicar que en el dataset de reseñas hay errores de formato en esta clave foránea, que puede estar alterando la cantidad real de libros. Al repetir la obtención de los libros distintos considerando los espacios en blanco

In [ ]:
reviews_df['ISBN'].str.strip().unique().shape[0]

340553

Se observa que en efecto existían duplicados por esta causa. En el dataset original de libros

In [ ]:
books_df['ISBN'].str.strip().unique().shape[0]

271359

Occure también un caso, aunque las ocurrencias son ínfimas en comparación con los otros registros que no cruzaron, lo que puede ser indicativo de otros problemas de consistencia del dato

## Análisis del dataset de GoodReads provisto



La fuente consiste en un solo conjunto de datos, que puede cargarse correctamente sin necesidad de configuración adicional, y sin la aparente presencia de valores erróneos

In [ ]:
good_reads_df = pd.read_csv(path + 'GoodReads.csv')

Está compuesto por

In [ ]:
good_reads_df.shape

(8468, 21)

filas y columnas, respectivamente. Las últimas representan información acerca de los libros, como título o número de páginas, un enlace a información del autor y métricas de las estadísticas del libro en la plataforma:

In [ ]:
for c in good_reads_df.columns:
  print(c)

book_id_title
book_id
cover_image_uri
book_title
book_series
book_settings
book_characters
book_language
year_first_published
authorlink
author
num_pages
format
genres
people_curr_read
peop_want_to_read
num_ratings
num_reviews
average_rating
rating_distribution
awards


Una inspección superficial de los registros muestra que algunas columnas presentan formatos complejos o anidados, como las listas de géneros, personajes y premios y la distribución de ratings

In [ ]:
good_reads_df.dropna().head(5)

,book_id_title,book_id,cover_image_uri,book_title,book_series,book_settings,book_characters,book_language,year_first_published,authorlink,...,num_pages,format,genres,people_curr_read,peop_want_to_read,num_ratings,num_reviews,average_rating,rating_distribution,awards
8,174465.Il_cavaliere_inesistente,174465,https://images-na.ssl-images-amazon.com/images...,Il cavaliere inesistente,['I nostri antenati'],['France'],"['Agilulfo', 'Charlemagne', 'Rambaldo', 'Gurdu...",Italian,1959.0,https://www.goodreads.com/author/show/155517.I...,...,126.0,Paperback,"['Fiction', 'Classics', 'Fantasy', 'Italian Li...",465,"8,944",23646,1071,3.83,"{'5 Stars': 6356, '4 Stars': 9385, '3 Stars': ...",[]
17,6539.The_Body_Farm,6539,https://images-na.ssl-images-amazon.com/images...,The Body Farm,['Kay Scarpetta'],"['Black Mountain, North Carolina', 'Virginia',...","['Kay Scarpetta', 'Pete Marino', 'Lucy Farinel...",English,1994.0,https://www.goodreads.com/author/show/1025097....,...,351.0,Paperback,"['Mystery', 'Crime', 'Fiction', 'Thriller', 'M...","1,276",15.7k,77953,1666,4.07,"{'5 Stars': 27666, '4 Stars': 31698, '3 Stars'...",[]
30,44492285-dune-messiah,44492285,https://images-na.ssl-images-amazon.com/images...,Dune Messiah,"['Dune', 'Dune Universe']",['Arrakis'],"['Stilgar', 'Duncan Idaho', 'Scytale', 'Paul A...",English,1969.0,https://www.goodreads.com/author/show/58.Frank...,...,336.0,Mass Market Paperback,"['Science Fiction', 'Fiction', 'Fantasy', 'Cla...",27.7k,160k,359097,22205,3.89,"{'5 Stars': 97182, '4 Stars': 149902, '3 Stars...",[]
40,657034.Heartsick,657034,https://images-na.ssl-images-amazon.com/images...,Heartsick,['Archie Sheridan & Gretchen Lowell'],"['Portland, Oregon', 'Oregon']","['Archie Sheridan', 'Susan Ward', 'Gretchen Lo...",English,2007.0,https://www.goodreads.com/author/show/48557.Ch...,...,326.0,Hardcover,"['Mystery', 'Thriller', 'Fiction', 'Crime', 'M...",637,46.4k,42387,3260,3.93,"{'5 Stars': 13887, '4 Stars': 16103, '3 Stars'...",[]
47,21874813-as-chimney-sweepers-come-to-dust,21874813,https://images-na.ssl-images-amazon.com/images...,As Chimney Sweepers Come to Dust,['Flavia de Luce'],"['Toronto, Ontario']",['Flavia de Luce'],English,2015.0,https://www.goodreads.com/author/show/1074866....,...,392.0,Hardcover,"['Mystery', 'Fiction', 'Historical Fiction', '...",847,20.2k,28695,3576,3.97,"{'5 Stars': 8241, '4 Stars': 12565, '3 Stars':...",['Goodreads Choice Award Nominee for Mystery &...


También demuestra la existencia de valores faltantes, al observar el detalle

In [ ]:
compute_nas(good_reads_df)

,Cantidad,Porciento
book_id_title,0,0.000000
book_id,0,0.000000
cover_image_uri,0,0.000000
book_title,0,0.000000
book_series,5762,0.680444
book_settings,5329,0.629310
book_characters,5238,0.618564
book_language,910,0.107463
year_first_published,15,0.001771
authorlink,0,0.000000


Destacan columnas con amplia proporción de valores faltantes, como los personajes y la serie a la que pertenece el libro